<a href="https://colab.research.google.com/github/quinzit/Master-project/blob/main/PDF_to_dataset_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.1 MB/s eta 0:00:00


In [24]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)


In [25]:
import pdfplumber
import pandas as pd
import os

In [26]:
from google.colab import files
uploaded = files.upload()


Saving Test.zip to Test (2).zip


In [27]:
import zipfile
import os

# Extract uploaded ZIP file
zip_path = "Test.zip"  # make sure this matches the file you upload
extract_dir = "pdfs"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Files extracted to:", extract_dir)


✅ Files extracted to: pdfs


In [28]:
import os

print("📁 Files inside extracted folder:")
for root, dirs, files in os.walk("pdfs"):
    for file in files:
        print(os.path.join(root, file))


📁 Files inside extracted folder:
pdfs/Test/order-O2025-3238.pdf
pdfs/Test/order-O2025-3248.pdf
pdfs/Test/order-O2025-3244.pdf


In [31]:
import pdfplumber
import os

pdf_dir = "pdfs/Test"

for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        full_path = os.path.join(pdf_dir, filename)
        print(f"\n📄 Inspecting {filename}...\n" + "-"*50)

        with pdfplumber.open(full_path) as pdf:
            page = pdf.pages[0]

            # Print raw extracted text
            print("📜 Raw Text:")
            print(page.extract_text())

            # Print all extracted tables
            tables = page.extract_tables()
            if not tables:
                print("\n❌ No tables found.")
            else:
                for i, table in enumerate(tables):
                    print(f"\n✅ Table {i+1} (first 5 rows):")
                    for row in table[:5]:
                        print(row)



📄 Inspecting order-O2025-3238.pdf...
--------------------------------------------------
📜 Raw Text:
Order confirmation O2025-3238
2204 - THAI ASIAN FOOD STORE D.O.O Date 03-04-2025
Uros Kos Order ID O2025-3238
Tugomerjeva Ulica 35 Reference Order 6276
1000 Ljubljana
Slovenia
thaiasianfoodstore@gmail.com
Product code Name Amount Price Total
1350 / 01-2026 L.R TB Bean Vermicelli 100x120g TH 3 € 0,00 € 0,00
1357 / 01-2026 L.R TB Dried Rice Noodle 100x120g TH 3 € 0,00 € 0,00
1351 / 01-2026 L.R TB Rice Vermicelli 100x130g TH 3 € 0,00 € 0,00
2922 / 07-2027 Chilli Powder 50x100g TH 1 € 0,00 € 0,00
2923 / 07-2026 DOUBLE PANDA Dehydrated Small Chilli Without Stem 50x100g TH 2 € 0,00 € 0,00
4901 / 02-2026 AROY-D coconut milk 12x1000ml TH 33 € 0,00 € 0,00
1508A / 12-2025 SAM YANG hot chicken ramen CarboMulti 8x5x130g KR 22 € 0,00 € 0,00
4282 / 05-2026 LGM Crispy chilli in oil 24x210g CN 5 € 0,00 € 0,00
3404 / 04-2026 LGM Crispy Chilli in Oil 12x670g CN 2 € 0,00 € 0,00
Total € 0,00
BTW (10%) € 0,

In [34]:
import pdfplumber
import pandas as pd
import re

pdf_dir = "pdfs/Test"  # adjust if your ZIP uses a different subfolder
all_data = []

for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        full_path = os.path.join(pdf_dir, filename)
        with pdfplumber.open(full_path) as pdf:
            page = pdf.pages[0]
            lines = page.extract_text().split("\n")

            # 🔎 Extract date
            extracted_date = None
            for line in lines:
                if "Date" in line:
                    parts = line.split()
                    for part in parts:
                        if re.match(r"\d{2}-\d{2}-\d{4}", part):
                            extracted_date = part
                            break

            # 📊 Find product rows
            table_started = False
            rows = []
            for line in lines:
                if line.strip().startswith("Product code"):
                    table_started = True
                    continue
                if table_started:
                    if line.strip().startswith("Total"):
                        break
                    rows.append(line.strip())

            # 🧾 Parse each product row
            for row in rows:
                try:
                    # Remove trailing price data
                    parts = row.rsplit("€", 2)[0].strip().rsplit(" ", 1)
                    rest = parts[0].strip()
                    amount = int(parts[1].strip())

                    # Match product code formats:
                    # 1508A / 12-2025, 8931 / 03-2025, or 3685
                    match = re.match(r"^([A-Z0-9]+(?:\s*/\s*\d{2,4}-\d{4})?)\s+(.*)", rest)
                    if match:
                        product_code = match.group(1).replace(" ", "")
                        name = match.group(2).strip()
                    else:
                        tokens = rest.split(" ", 1)
                        product_code = tokens[0].strip()
                        name = tokens[1].strip() if len(tokens) > 1 else ""

                    all_data.append({
                        "Product code": product_code,
                        "Name": name,
                        "Amount": amount,
                        "Date": extracted_date,
                        "File": filename
                    })
                except Exception as e:
                    print(f"⚠️ Skipped row due to error: {row}\n➡️ Reason: {e}")
#export to csv file:
if all_data:
    final_df = pd.DataFrame(all_data)
    output_file = "extracted_data.xlsx"
    final_df.to_excel(output_file, index=False)

    from google.colab import files
    files.download(output_file)
    print("✅ Done! Excel file is ready to download.")
else:
    print("❌ No data extracted. Please check PDF content.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done! Excel file is ready to download.
